# Otimizando os hiper-parâmetros c/ Random Search

## Importando as libs

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

## 1. Aquisição dos dados

In [2]:
number_mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = number_mnist.load_data()

In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## 2. Pré-processamento

In [4]:
X_valid, X_train = X_train[:5000] / 255., X_train[5000:] / 255.
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test / 255.

In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_valid.shape)
print(y_valid.shape)

(55000, 28, 28)
(55000,)
(10000, 28, 28)
(10000,)
(5000, 28, 28)
(5000,)


## 3. Construindo e treinando o modelo

In [4]:
def build_model(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape= input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))    
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]) 
    return model

In [5]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [6]:
model.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 

Epoch 1/30
1719/1719 [==============================] - 5s 2ms/step - loss: 1.5201 - accuracy: 0.6130 - val_loss: 0.5938 - val_accuracy: 0.8656
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5518 - accuracy: 0.8659 - val_loss: 0.4298 - val_accuracy: 0.8918
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4307 - accuracy: 0.8863 - val_loss: 0.3683 - val_accuracy: 0.9024
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3742 - accuracy: 0.8992 - val_loss: 0.3346 - val_accuracy: 0.9108
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3472 - accuracy: 0.9049 - val_loss: 0.3117 - val_accuracy: 0.9158
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3253 - accuracy: 0.9107 - val_loss: 0.2950 - val_accuracy: 0.9212
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3073 - accuracy: 0.9158 - val_loss: 0.2822 - val_accuracy:

## 4. Otimizando os parâmetros com o RandomizedSearchCV

In [7]:
param_distribs = {
    "n_hidden": [1, 2, 3, 4],
    "n_neurons": np.arange(268, 400).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [8]:
rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10), 
                             keras.callbacks.ModelCheckpoint("melhor_modelo_mlp_hiperparameters2.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/30
1146/1146 [==============================] - 6s 4ms/step - loss: 2.2516 - accuracy: 0.1772 - val_loss: 1.8456 - val_accuracy: 0.5880
Epoch 2/30
1146/1146 [==============================] - 4s 4ms/step - loss: 1.7573 - accuracy: 0.6391 - val_loss: 1.4657 - val_accuracy: 0.7416
Epoch 3/30
1146/1146 [==============================] - 5s 4ms/step - loss: 1.4095 - accuracy: 0.7468 - val_loss: 1.1925 - val_accuracy: 0.7892
Epoch 4/30
1146/1146 [==============================] - 5s 4ms/step - loss: 1.1592 - accuracy: 0.7942 - val_loss: 1.0046 - val_accuracy: 0.8144
Epoch 5/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.9860 - accuracy: 0.8141 - val_loss: 0.8748 - val_accuracy: 0.8322
Epoch 6/30
1146/1146 [==============================] - 5s 4ms/step - loss: 0.8647 - accuracy: 0.8276 - val_loss: 0.7823 - val_accuracy: 0.8436
Epoch 7/30
1146/1146 [==============================] - 5s 5ms/step - loss:

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.0004497801025474755, n_hidden=1, n_neurons=394; total time= 2.5min
Epoch 1/30
1146/1146 [==============================] - 4s 3ms/step - loss: 2.1865 - accuracy: 0.2085 - val_loss: 1.8227 - val_accuracy: 0.6070
Epoch 2/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.7262 - accuracy: 0.6453 - val_loss: 1.4499 - val_accuracy: 0.7386
Epoch 3/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.3851 - accuracy: 0.7444 - val_loss: 1.1836 - val_accuracy: 0.7848
Epoch 4/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.1457 - accuracy: 0.7826 - val_loss: 1.0015 - val_accuracy: 0.8082
Epoch 5/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.9776 - accuracy: 0.8034 - val_loss: 0.8752 - val_accuracy: 0.8270
Epoch 6/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.8604 - accuracy: 0.8226 - val_loss: 0.7846 - val_accuracy: 0.8362
Epoch 7/30
1146/1146 [======================

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.0004497801025474755, n_hidden=1, n_neurons=394; total time= 1.7min
Epoch 1/30
1146/1146 [==============================] - 4s 3ms/step - loss: 2.1372 - accuracy: 0.3095 - val_loss: 1.7635 - val_accuracy: 0.6092
Epoch 2/30
1146/1146 [==============================] - 4s 3ms/step - loss: 1.6757 - accuracy: 0.6364 - val_loss: 1.4026 - val_accuracy: 0.7376
Epoch 3/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.3538 - accuracy: 0.7398 - val_loss: 1.1475 - val_accuracy: 0.7890
Epoch 4/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.1208 - accuracy: 0.7826 - val_loss: 0.9723 - val_accuracy: 0.8188
Epoch 5/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.9591 - accuracy: 0.8076 - val_loss: 0.8499 - val_accuracy: 0.8350
Epoch 6/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.8454 - accuracy: 0.8290 - val_loss: 0.7614 - val_accuracy: 0.8474
Epoch 7/30
1146/1146 [======================

...

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.015114765647573948, n_hidden=1, n_neurons=287; total time= 1.4min
Epoch 1/30
1146/1146 [==============================] - 3s 2ms/step - loss: 1.0627 - accuracy: 0.7310 - val_loss: 0.3564 - val_accuracy: 0.9062
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3602 - accuracy: 0.9004 - val_loss: 0.2910 - val_accuracy: 0.9224
Epoch 3/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3012 - accuracy: 0.9146 - val_loss: 0.2608 - val_accuracy: 0.9278
Epoch 4/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.2641 - accuracy: 0.9266 - val_loss: 0.2338 - val_accuracy: 0.9352
Epoch 5/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.2372 - accuracy: 0.9336 - val_loss: 0.2163 - val_accuracy: 0.9414
Epoch 6/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.2191 - accuracy: 0.9383 - val_loss: 0.1985 - val_accuracy: 0.9476
Epoch 7/30
1146/1146 [=======================

c:\users\alexw\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END learning_rate=0.015114765647573948, n_hidden=1, n_neurons=287; total time=  59.6s
Epoch 1/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.9152 - accuracy: 0.7517 - val_loss: 0.2173 - val_accuracy: 0.9386
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2064 - accuracy: 0.9401 - val_loss: 0.1445 - val_accuracy: 0.9612
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1354 - accuracy: 0.9606 - val_loss: 0.1221 - val_accuracy: 0.9648
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1013 - accuracy: 0.9693 - val_loss: 0.0973 - val_accuracy: 0.9734
Epoch 5/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0800 - accuracy: 0.9754 - val_loss: 0.0888 - val_accuracy: 0.9734
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0633 - accuracy: 0.9806 - val_loss: 0.0869 - val_accuracy: 0.9750
Epoch 7/30
1719/1719 [=======================

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x0000029AD5C33F70>,
                   param_distributions={'learning_rate': [0.012244261547686505,
                                                          0.02392460863595287,
                                                          0.020111098777089338,
                                                          0.004762211431961521,
                                                          0.010392905078550537,
                                                          0.0018603043619865288,
                                                          0.0017369166054379242,
                                                          0.0004955137402474184,
                                                          0.0043300122358824...
                                                          0.0009051629413466672,
                                                 

## 5. Avaliando o melhor modelo

In [9]:
best_model = keras.models.load_model("melhor_modelo_mlp_hiperparameters2.h5")

In [10]:
best_model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_110 (Dense)            (None, 362)               284170    
_________________________________________________________________
dense_111 (Dense)            (None, 362)               131406    
_________________________________________________________________
dense_112 (Dense)            (None, 362)               131406    
_________________________________________________________________
dense_113 (Dense)            (None, 362)               131406    
_________________________________________________________________
dense_114 (Dense)            (None, 10)                3630      
Total params: 682,018
Trainable params: 682,018
Non-trainable params: 0
_______________________________________________

In [11]:
# Meta: Acurácia: 0.9798 e Kappa:  0.9775466807563838
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test,y_pred))
print('Kappa: ', cohen_kappa_score(y_test,y_pred))
print('Matriz de confusão: \n', confusion_matrix(y_test,y_pred))

Acurácia:  0.9793
Kappa:  0.9769906167735203
Matriz de confusão: 
 [[ 965    0    4    1    1    3    1    1    3    1]
 [   0 1127    2    0    0    1    2    0    3    0]
 [   1    4 1013    2    2    0    1    5    4    0]
 [   0    1    7  992    0    2    0    3    3    2]
 [   0    0    5    0  970    0    1    3    0    3]
 [   2    0    0    9    2  869    4    1    3    2]
 [   4    2    1    1    4    8  935    0    3    0]
 [   1    5    7    2    1    0    0 1003    4    5]
 [   4    0    2    6    3    4    4    2  946    3]
 [   1    6    0    9   11    5    1    2    1  973]]
